# EDA - preping the industry data

In [41]:
import os.path as osp

In [1]:
import pyspark 
sc = pyspark.sql.SparkSession.Builder()\
        .master('yarn')\
        .config('spark.executor.instances', '1')\
        .appName('hive-matt')\
        .enableHiveSupport()\
        .getOrCreate()

In [2]:
sc

In [3]:
sc.sql('SHOW TABLES').collect()

[Row(database='default', tableName='pac_spending', isTemporary=False),
 Row(database='default', tableName='pac_spending_by_type', isTemporary=False),
 Row(database='default', tableName='pq_crp_cands18', isTemporary=False),
 Row(database='default', tableName='pq_crp_cmtes18', isTemporary=False),
 Row(database='default', tableName='pq_crp_expends18', isTemporary=False),
 Row(database='default', tableName='pq_crp_indivs18', isTemporary=False),
 Row(database='default', tableName='pq_crp_pac_other18', isTemporary=False),
 Row(database='default', tableName='pq_crp_pacs18', isTemporary=False),
 Row(database='default', tableName='pq_pp_bills', isTemporary=False),
 Row(database='default', tableName='pq_pp_sponsors', isTemporary=False),
 Row(database='default', tableName='pq_pp_titles', isTemporary=False)]

In [4]:
sc.sql('SELECT * FROM pq_crp_cands18').limit(10).show()

+-----+---------+---------+--------------------+-----+------------+----------+--------+---------+------+---------+------+
|cycle|feccandid|      cid|          firstlastp|party|distidrunfor|distidcurr|currcand|cyclecand|crpico|recipcode|nopacs|
+-----+---------+---------+--------------------+-----+------------+----------+--------+---------+------+---------+------+
| 2018|H6FL08221|N00028542|       D J Mauro (I)|    I|        FL08|          |        |         |      |       3N|      |
| 2018|H6NC09200|N00035491|     Mark Harris (R)|    R|        NC09|          |        |         |     O|       RO|      |
| 2018|H4NE01163|N00035619| Dennis Crawford (D)|    D|        NE01|          |        |        Y|     C|       DL|      |
| 2018|H0IN01127|N00041294| Arturas Kerelis (D)|    D|        IN01|          |        |         |      |       DN|      |
| 2018|H8AL02171|N00041295|     Barry Moore (R)|    R|        AL02|          |        |        Y|     C|       RL|      |
| 2018|H8AL04078|N000412

Does the individual table not have industry??

In [5]:
sc.sql('SELECT * FROM pq_crp_indivs18').limit(5).show()

+-----+-------------------+------------+------------------+---------+-------------------+------+--------+----------+------+------+---------------+-----+-----+---------+----+---------+---------+------+------------------+------------+-------------------+------+
|cycle|         fectransid|   contribid|           contrib|  recipid|            orgname|ultorg|realcode|      date|amount|street|           city|state|  zip|recipcode|type|   cmteid|  otherid|gender|         microfilm|  occupation|           employer|source|
+-----+-------------------+------------+------------------+---------+-------------------+------+--------+----------+------+------+---------------+-----+-----+---------+----+---------+---------+------+------------------+------------+-------------------+------+
| 2018|2103020171461527012|j1002215457 |   SCHULTZ, ANDREW|N00027522| 2 Building Lbj Ltd|      |   Y4000|2017-07-14|  2000|      |        ROWLETT|   TX|75030|       RI| 15 |C00494229|         |     M|201710200200354160| 

In [6]:
sc.sql('SELECT realcode, sum(amount), count(contribid) FROM pq_crp_indivs18 GROUP BY realcode ').count()

470

In [7]:
sc.sql('SELECT realcode, sum(amount) AS money, count(contribid) FROM pq_crp_indivs18 GROUP BY realcode ORDER BY money DESC').collect()

[Row(realcode='Z9500', money=307821522, count(contribid)=2563962),
 Row(realcode='Y4000', money=227928460, count(contribid)=571205),
 Row(realcode='X1200', money=186889529, count(contribid)=972393),
 Row(realcode='K1000', money=94909387, count(contribid)=161371),
 Row(realcode='F2700', money=65099756, count(contribid)=4112),
 Row(realcode='F2600', money=57948077, count(contribid)=14944),
 Row(realcode='J1200', money=55168158, count(contribid)=1102926),
 Row(realcode='Y1000', money=49244720, count(contribid)=279555),
 Row(realcode='F2100', money=49068817, count(contribid)=48934),
 Row(realcode='Z9000', money=48915983, count(contribid)=10148),
 Row(realcode='F4000', money=46599518, count(contribid)=39289),
 Row(realcode='J1100', money=40848862, count(contribid)=71784),
 Row(realcode='F4100', money=38014293, count(contribid)=15586),
 Row(realcode='LB100', money=34695187, count(contribid)=208341),
 Row(realcode='JE300', money=33903445, count(contribid)=8861),
 Row(realcode='K2000', money=3

^Looks like all individuals have an industry, which is impressive given the number of blanks. Maybe one of the codes is unknown. Haven't found a translation table yet for realcodes 

criteria for filter individuals:
    
* Realcode not like Z9* (these are noncontributions)
* Limit types to 10, 11, 15, 15E, 15J, 22Y
* Remember that Type 10 is soft money before 2004 and Levin Funds or outside spending 2004+
* RecipID like N* limits to candidates
* Source <> P/PAC excludes contributions to PACs other than leadership PACs

****MUST COME BACK TO AND JOIN WITH PAC CANDIDATE CONTRIBUTIONS****

In [8]:
sc.sql('SELECT * FROM pq_crp_cmtes18').limit(10).show()

+-----+---------+--------------------+--------------------+--------------------+---------+---------+---------+-----+--------+------+---------+-------+------+
|cycle|   cmteid|            pacshort|           affiliate|              ultorg|  recipid|recipcode|feccandid|party|primcode|source|sensitive|foreign|active|
+-----+---------+--------------------+--------------------+--------------------+---------+---------+---------+-----+--------+------+---------+-------+------+
| 2018|C00000018|                    |                    |                    |C00000018|         |H8TX22313|     |        |      |         |  false|     0|
| 2018|C00000059|      Hallmark Cards|                    |      Hallmark Cards|C00000059|       PB|         |     |   C1400| WebAM|        n|  false|     1|
| 2018|C00000422|American Medical ...|American Medical ...|American Medical ...|C00000422|       PB|         |     |   H1100| AFP88|        n|  false|     1|
| 2018|C00000489| Teamsters Local 886|     Teamsters

In [9]:
sc.sql('SELECT primcode FROM pq_crp_cmtes18 GROUP BY primcode').count()

408

In [10]:
sc.sql('SELECT primcode FROM pq_crp_cmtes18 GROUP BY primcode').show(10)

+--------+
|primcode|
+--------+
|   C2100|
|   C1100|
|   T3100|
|   L1300|
|   T9000|
|   j7000|
|   J9000|
|   J1000|
|   M2400|
|   G6700|
+--------+
only showing top 10 rows



In [11]:
sc.sql('SELECT * FROM pq_crp_expends18').limit(1).show()

+-----+---+-----------------+----------+---------+--------------------+-------------+-------+------+----------+-----------+-----+---------+---------+------+----+------------------+-----+---------+-------+------+
|Cycle| ID|          TransID|CRPFilerid|Recipcode|            Pacshort| CRPRecipname|Expcode|Amount|      Date|       City|State|      Zip|CmteID_EF|Candid|Type|           Descrip|   PG|ElecOther|EntType|Source|
+-----+---+-----------------+----------+---------+--------------------+-------------+-------+------+----------+-----------+-----+---------+---------+------+----+------------------+-----+---------+-------+------+
| 2018|  1|SB21B-11865-10000| C00460147|       DP|DNC/Travel Offset...|W Los Angeles|    F30| 554.0|2017-01-18|Los Angeles|   CA|90024    |         |      |    |Lodging & Catering|     |         |    ORG|  @new|
+-----+---+-----------------+----------+---------+--------------------+-------------+-------+------+----------+-----------+-----+---------+---------+---

In [12]:
sc.sql('SELECT * FROM pq_crp_pacs18').limit(10).show()

+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
|cycle|           fecrecno|    pacid|      cid|amount|      date|realcode|type| di|feccandid|
+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
| 2018|4031520181518958016|C00150797|N00035527|1000.0|2018-02-06|   K1200| 24K|  D|H4AR04048|
| 2018|4021520181504727622|C00343459|N00028152|5000.0|2017-12-27|   H1130| 24K|  D|H6CA22125|
| 2018|4062120181570992000|C00005249|N00035509|3000.0|2018-05-18|   F3300| 24K|  D|H4IA03115|
| 2018|4121120171484669562|C00449165|N00003689|1000.0|2017-11-08|   M3100| 24K|  D|H8OH01043|
| 2018|4101220171458221652|C00171843|N00002408|5000.0|2017-09-21|   F3400| 24K|  D|H2SC02042|
| 2018|4051820181565917536|C00035675|N00031933|1000.0|2018-04-23|   H4300| 24K|  D|H0MA10082|
| 2018|4072420171429498467|C00035451|N00034584|2000.0|2017-06-27|   LT100| 24K|  D|S2ME00158|
| 2018|4112020171465723593|C00033779|N00035346|1000.0|2017-1

In [13]:
sc.sql('SELECT * FROM pq_crp_pacs18').count()

191318

need to joing the  pq_crp_pacs18 to the candidate table to get party detail. I think realcode and primcode are the same thing...
* To calculate direct contributions to candidates, you limit to DI = D

In [15]:
sc.sql('SELECT sum(amount) FROM pq_crp_pacs18 where di == "D"').limit(10).show()

+------------+
| sum(amount)|
+------------+
|3.21276746E8|
+------------+



In [16]:
sc.sql('SELECT pacid,t2.cid, amount, date, realcode,type, di,t1.feccandid,firstlastp,party \
       FROM pq_crp_pacs18 t1 \
       INNER JOIN pq_crp_cands18 t2 \
       ON t1.feccandid= t2.feccandid \
       WHERE di == "D" AND \
       realcode NOT LIKE "z9*%" AND \
       realcode NOT LIKE "z4*%"').limit(10).show()

#realcode is industry, this filters out transfers and joint fund raising committees

+---------+---------+------+----------+--------+----+---+---------+--------------------+-----+
|    pacid|      cid|amount|      date|realcode|type| di|feccandid|          firstlastp|party|
+---------+---------+------+----------+--------+----+---+---------+--------------------+-----+
|C00150797|N00035527|1000.0|2018-02-06|   K1200| 24K|  D|H4AR04048| Bruce Westerman (R)|    R|
|C00343459|N00028152|5000.0|2017-12-27|   H1130| 24K|  D|H6CA22125|  Kevin McCarthy (R)|    R|
|C00005249|N00035509|3000.0|2018-05-18|   F3300| 24K|  D|H4IA03115|     David Young (R)|    R|
|C00449165|N00003689|1000.0|2017-11-08|   M3100| 24K|  D|H8OH01043|    Steve Chabot (R)|    R|
|C00171843|N00002408|5000.0|2017-09-21|   F3400| 24K|  D|H2SC02042| James E Clyburn (D)|    D|
|C00035675|N00031933|1000.0|2018-04-23|   H4300| 24K|  D|H0MA10082|    Bill Keating (D)|    D|
|C00035451|N00034584|2000.0|2017-06-27|   LT100| 24K|  D|S2ME00158|  Bruce Poliquin (R)|    R|
|C00033779|N00035346|1000.0|2017-10-24|   F4100| 2

Need industry names!

In [19]:
import re
import pandas as pd
#crp_industry_names = ast.literal_eval(open("crp_industry_names.txt", "r").read
                                      
crp_industry_names = open("crp_industry_names.txt", "r").read()#.split('\t')
crp_industry_names
crp_industry_names = crp_industry_names.splitlines()


crp_industry_names = [re.split(r'\t+',x) for x in crp_industry_names]
crp_industry_names =pd.DataFrame(crp_industry_names)

crp_industry_names.columns=("Catcode","Catname","Catorder","Industry","Sector","Sector Long")
crp_industry_names= crp_industry_names[1:]
crp_industry_names.head()

,Catcode,Catname,Catorder,Industry,Sector,Sector Long
1,A0000,Agriculture,A11,Misc Agriculture,Agribusiness,Agribusiness
2,A1000,Crop production & basic processing,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
3,A1100,Cotton,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
4,A1200,Sugar cane & sugar beets,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
5,A1300,Tobacco & Tobacco products,A02,Tobacco,Agribusiness,Agribusiness


In [20]:
ind_names = sc.broadcast(crp_industry_names)

AttributeError: 'SparkSession' object has no attribute 'broadcast'

In [21]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
crp_industry_names = sqlCtx.createDataFrame(data = crp_industry_names)

In [22]:
crp_industry_names

DataFrame[Catcode: string, Catname: string, Catorder: string, Industry: string, Sector: string, Sector Long: string]

In [35]:
import pyspark
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, FloatType, BooleanType, DateType
import time
import datetime


IndnamesSchemaStr = [
'''Catcode
Unique indentifier for each industry. Same is Prim_code or realcode
Text (5)
CRP''',

'''Catname
Name for each industry
Text (9)
FEC''',

'''Catorder
Grouping similar industries together with a code, NOT an abbreviation of Catcode
Text (3)
CRP''',

'''Industry
Not really a one to one with any level of NAICS
Text (25)
CRP''',
    
'''Sector
A level up from Industry 
Text (15)
CRP''',

'''SectorLong
A level up from Industry 
Text (40)
CRP''' 
]

def now():
    return datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S.%f')

In [36]:
from open_secrets_schema import Schema
cands18Schema = Schema(IndnamesSchemaStr)
cands18 = pd.read_csv('crp_industry_names.txt', quotechar='"', delimiter='\t', 
                      parse_dates=cands18Schema.get_dt_colnames(), infer_datetime_format=True,
                      header=0, dtype=cands18Schema.get_pandas_schema(), names=cands18Schema.column_names)

cands18.head(3)

,Catcode,Catname,Catorder,Industry,Sector,SectorLong
0,A0000,Agriculture,A11,Misc Agriculture,Agribusiness,Agribusiness
1,A1000,Crop production & basic processing,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
2,A1100,Cotton,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness


In [37]:
!hdfs dfs -ls /staging

2018-10-20 16:09:27,114 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxrwxrwx   - matt supergroup          0 2018-10-18 05:33 /staging/bills
drwxr-xr-x   - matt supergroup          0 2018-10-20 16:03 /staging/industry_names


In [38]:
!hdfs dfs -rm -r -f /staging/industry_names

2018-10-20 16:09:38,073 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted /staging/industry_names


In [32]:
#!hdfs dfs -mkdir -p /staging/industry_names

2018-10-20 16:03:11,532 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [33]:
#sc.sql('CREATE TABLE industry_names (Catcode STRING, Catname STRING, Catorder STRING, Industry STRING, Sector STRING, Sector_Long STRING)')

In [34]:
#import findspark
#findspark.init()
#import pyspark
#from pyspark.sql import HiveContext
#
#sqlCtx= HiveContext(sc)
#
#spark_df = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("./data/documents_topics.csv")
#spark_df.registerTempTable("my_table")
#
#sqlCtx.sql("CREATE TABLE my_table_2 AS SELECT * from my_table")

In [77]:
!head -n 2 crp_industry_names.txt

Remove header with `tail -n +2 {filename}`

In [78]:
!tail -n +2 crp_industry_names.txt > crp_industry_names_noheader.txt

In [79]:
def etl_text_file(text_file_path, schema_str, table_name, data_source_name, column_datetime_formats=None,
                  csv_quotechar='|', csv_delimiter=','):
    schema = Schema(schema_str)
    schema.register_column_datetime_formats(column_datetime_formats)
    textFileName = osp.split(text_file_path)[1]
    stagingFolder = f'/staging/{table_name}'
    !hdfs dfs -rm -r -f {stagingFolder}/{textFileName}
    !hdfs dfs -mkdir -p {stagingFolder}
    !hdfs dfs -put {text_file_path} {stagingFolder}/{textFileName}
    !hdfs dfs -chmod -R 777 {stagingFolder}
    with open(f'{table_name}.sql', 'w') as f:
        f.write(schema.hive_ddl(tableName=table_name, externalTableLocation=stagingFolder,
                                csv_quotechar=csv_quotechar, csv_delimiter=csv_delimiter))
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f {table_name}.sql
    
    # Convert to the proper schema and write as parquet
    finalTableName = f'pq_{data_source_name}_{table_name}'
    !hdfs dfs -chmod -R 777 '/user/hive/warehouse'
    with open(f'{finalTableName}.sql', 'w') as f:
        f.write(schema.hive_ddl_parquet(tableName=finalTableName, originalTableName=table_name))
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f {finalTableName}.sql
    
    # clean up -- drop the intermediate (text file) table and delete staged data from HDFS
    sc.sql(f'DROP TABLE {table_name}')
    !hdfs dfs -rm -r -f {stagingFolder}
    
    print('done')

In [80]:
etl_text_file(text_file_path='crp_industry_names_noheader.txt',
              schema_str=IndnamesSchemaStr,
              table_name='industry_names',
              data_source_name='crp',
              csv_quotechar='\\"',
              csv_delimiter='\\t')

2018-10-20 17:33:09,080 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-20 17:33:11,115 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-20 17:33:13,148 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-20 17:33:15,389 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Connecting to jdbc:hive2://cap1:10000
2018-10-20 17:33:17 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-20 17:33:17 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-20 17:33:17 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Trans

Connecting to jdbc:hive2://cap1:10000
2018-10-20 17:33:21 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-20 17:33:21 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-20 17:33:21 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://cap1:10000> 
0: jdbc:hive2://cap1:10000> DROP TABLE IF EXISTS pq_crp_industry_names;
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.112 seconds)
0: jdbc:hive2://cap1:10000>         
0: jdbc:hive2://cap1:10000> CREATE TABLE pq_crp_industry_names 
0.273595 from industry_names' MMENT 'Auto-generated schema at 2018-10-20 17:33:2 
0: jdbc:hive2://cap1:10000> STORED AS PARQUET 
0: jdbc:hive2://cap1:10000> AS SELECT
0: jdbc:hive2://cap1:10000>     `Catcode` AS `Catcode`,
0: jdbc:hive2://cap1:10000>     `Catname` AS `Catname`,

OK, we get a "filesystem closed" exception around that point in the logs. no idea why. but we can use Spark to load in the data, so we will do that.

In [81]:
!hdfs dfs -mkdir -p /staging/industry_names

2018-10-20 17:57:27,634 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [82]:
!hdfs dfs -put crp_industry_names_noheader.txt /staging/industry_names/crp_industry_names_noheader.txt

2018-10-20 18:04:04,563 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [83]:
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f industry_names.sql

Connecting to jdbc:hive2://cap1:10000
2018-10-20 18:04:32 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-20 18:04:32 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-20 18:04:32 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://cap1:10000> 
0: jdbc:hive2://cap1:10000> DROP TABLE IF EXISTS industry_names;
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.113 seconds)
0: jdbc:hive2://cap1:10000>         
0: jdbc:hive2://cap1:10000> CREATE EXTERNAL TABLE industry_names
0: jdbc:hive2://cap1:10000> (
P',ch industry. Same is Prim_code or realcode, orig-datatype Text (5), source CR 
, orig-datatype Text (9), source FEC',me` STRING COMMENT 'Name for each industry 
(3), source CRP',ith a code, NOT an abbreviation of Catcode, orig-datatype Text  
ne with 

In [86]:
schema = Schema(IndnamesSchemaStr)

In [84]:
df = sc.sql('SELECT * FROM industry_names')

apply schema

In [90]:
newDF = sc.createDataFrame(df.rdd, schema=schema.get_spark_dtypes())

In [91]:
newDF.show(2)

+-------+--------------------+--------+--------------------+------------+------------+
|Catcode|             Catname|Catorder|            Industry|      Sector|  SectorLong|
+-------+--------------------+--------+--------------------+------------+------------+
|  A0000|         Agriculture|     A11|    Misc Agriculture|Agribusiness|Agribusiness|
|  A1000|Crop production &...|     A01|Crop Production &...|Agribusiness|Agribusiness|
+-------+--------------------+--------+--------------------+------------+------------+
only showing top 2 rows



In [92]:
newDF.schema

StructType(List(StructField(Catcode,StringType,true),StructField(Catname,StringType,true),StructField(Catorder,StringType,true),StructField(Industry,StringType,true),StructField(Sector,StringType,true),StructField(SectorLong,StringType,true)))

save to Hive

In [93]:
newDF.write.saveAsTable('pq_crp_industry_names', format='parquet', mode='overwrite')

update comments using the latter part of the 'pq_crp_industry_names.sql' file written by the `Schema` object

In [96]:
%cat pq_crp_industry_names.sql | tail -n 7


ALTER TABLE pq_crp_industry_names CHANGE `Catcode` `Catcode` STRING COMMENT 'Unique indentifier for each industry. Same is Prim_code or realcode';
ALTER TABLE pq_crp_industry_names CHANGE `Catname` `Catname` STRING COMMENT 'Name for each industry';
ALTER TABLE pq_crp_industry_names CHANGE `Catorder` `Catorder` STRING COMMENT 'Grouping similar industries together with a code, NOT an abbreviation of Catcode';
ALTER TABLE pq_crp_industry_names CHANGE `Industry` `Industry` STRING COMMENT 'Not really a one to one with any level of NAICS';
ALTER TABLE pq_crp_industry_names CHANGE `Sector` `Sector` STRING COMMENT 'A level up from Industry ';
ALTER TABLE pq_crp_industry_names CHANGE `SectorLong` `SectorLong` STRING COMMENT 'A level up from Industry ';


In [97]:
%cat pq_crp_industry_names.sql | tail -n 7 > pq_crp_industry_names_comments.sql

In [98]:
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f pq_crp_industry_names_comments.sql

Connecting to jdbc:hive2://cap1:10000
2018-10-20 18:09:31 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-20 18:09:31 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-20 18:09:31 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://cap1:10000> 
 or realcode';G COMMENT 'Unique indentifier for each industry. Same is Prim_code 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.276 seconds)
Catname` STRING COMMENT 'Name for each industry';dustry_names CHANGE `Catname` ` 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.103 seconds)
 an abbreviation of Catcode';ouping similar industries together with a code, NOT 
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.094 seconds)
`Industry` STRING COMMENT

In [102]:
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'DESCRIBE pq_crp_industry_names;'

Connecting to jdbc:hive2://cap1:10000
2018-10-20 18:10:23 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-20 18:10:23 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-20 18:10:23 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
+-------------+------------+-----------------------------------------------------------------------------------+--+
|  col_name   | data_type  |                                      comment                                      |
+-------------+------------+-----------------------------------------------------------------------------------+--+
| Catcode     | string     | Unique indentifier for each industry. Same is Prim_code or realcode               |
| Catname     | string     | Name for each industry                                                            |

clean up

In [103]:
!hdfs dfs -rm -r -f /staging/industry_names
!beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'DROP TABLE industry_names;'

2018-10-20 18:14:10,739 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted /staging/industry_names
Connecting to jdbc:hive2://cap1:10000
2018-10-20 18:14:12 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-20 18:14:12 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-20 18:14:12 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.202 seconds)
Beeline version 1.2.1.spark2 by Apache Hive
Closing: 0: jdbc:hive2://cap1:10000


verify all data got written:

In [134]:
sc.sql('SELECT COUNT(*) FROM pq_crp_industry_names').collect()

[Row(count(1)=459)]

In [135]:
!cat crp_industry_names.txt | wc -l

459


In [89]:
#!hdfs dfs -mkdir -p //cap1/user/jennifer/industry

In [44]:
#crp_industry_names.to_parquet('//cap1/user/jennifer/df.parquet.gzip', compression='gzip')

In [45]:
#from pyspark.sql import HiveContext
#
#sqlCtx= HiveContext(sc)
#
#df = sqlCtx.sql("SELECT * FROM parquet.`//cap1/user/jennifer/df.parquet`")